The idea of the visualization and the base code is taken from https://github.com/AIML-IfI/openset-imagenet-comparison. It was developed **by Palechor, Andres and Bhoumik, Annesha and Günther, Manuel** for the paper "**Large-Scale Open-Set Classification Protocols for ImageNet**" (Winter Conference on Applications of Computer Vision (WACV)) in 2023. You find the full citation in the readme file.

We have adapted it to fit our needs to visualize many different methods in one plot.

In [1]:
import argparse
import multiprocessing
import collections
import subprocess
import pathlib
import os, sys
import torch
import numpy

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

from collections import defaultdict

from matplotlib import pyplot, cm, colors
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MaxNLocator, LogLocator

from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import ticker
from matplotlib.lines import Line2D
from matplotlib.ticker import LogLocator, NullFormatter
from matplotlib import colors
import matplotlib.cm as cm

import yaml

THRESHOLDS = {1e-4: "$10^{-4}$",
              1e-3: "$10^{-3}$",
              1e-2: "$10^{-2}$",
              1e-1: "$10^{-1}$",
              0.2: "$0.2$",
              1: "$1$",
}



# get distinguishable colors
import matplotlib.cm
colors = matplotlib.cm.tab10(range(10))

COLORS = {
    "threshold": colors[0],
    "openmax": colors[8],
    "proser": colors[2],
    "evm": colors[3],
    "maxlogits": colors[5]
}

STYLES = {
    "entropic": "solid",
    "softmax": "solid",
    "garbage": "dotted",
    "p1": "dashed",
    "p2": "dotted",
    "p3": "solid"
}

NAMES = {
    "threshold": "Threshold",
    "openmax": "OpenMax",
    "proser": "PROSER",
    "evm": "EVM",
    "maxlogits": "MaxLogits",
    "entropic": "EOS",
    "softmax": "Softmax",
    "garbage": "Garbage",
    "p1": "P_1",
    "p2": "P_2",
    "p3": "P_3",
    1: "$P_1$",
    2: "$P_2$",
    3: "$P_3$"
}



In [ ]:
def load_scores(use_best, algorithm, loss, protocol ,method, eval, experiment_number):
    suffix = "best" if use_best else "curr"
    scores = defaultdict(lambda: defaultdict(dict))
    ground_truths = {}
    scr = "scores"

    score_file = f"{loss}_{algorithm}_{eval}_arr_{suffix}_{str(experiment_number)}.npz"

    if(os.path.exists(score_file)):
        results = numpy.load(score_file)
        scores[protocol][loss][algorithm] = results[scr]
        if(protocol not in ground_truths):
            ground_truths[protocol] = results["gt"].astype(int)
        else:
            assert numpy.all(results["gt"] == ground_truths[protocol])

    else:
      print("did not find the data")

    return scores, ground_truths

def plot_single_oscr(fpr, ccr, ax, loss, algorithm, scale, color = 'r'):
    linewidth = 0.9
    if scale == 'log':
        ax.set_xscale('log')
        ax.set_yscale('log')
        # Manual limits
        ax.set_ylim(0.09, 1)
        ax.set_xlim(8 * 1e-5, 1.4)
        # Manual ticks
        ax.xaxis.set_major_locator(LogLocator(base=10, numticks=100))
        locmin = ticker.LogLocator(base=10.0, subs=np.linspace(0, 1, 10, False), numticks=12)
        ax.xaxis.set_minor_locator(locmin)
        ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    elif scale == 'semilog':
        ax.set_xscale('log')
        # Manual limits
        ax.set_ylim(0.0, .8)
        ax.set_xlim(8 * 1e-5, 1.4)
        # Manual ticks
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))  # MaxNLocator(7))  #, prune='lower'))
        ax.xaxis.set_major_locator(LogLocator(base=10, numticks=10))
        locmin = ticker.LogLocator(base=10.0, subs=np.linspace(0, 1, 10, False), numticks=12)
        ax.xaxis.set_minor_locator(locmin)
        ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    else:
        ax.set_ylim(0.0, 0.8)
        # ax.set_xlim(None, None)
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))  # , prune='lower'))
    ax.plot(fpr,
            ccr,
            linestyle=STYLES[loss],
            color=color,
            linewidth=linewidth)  # marker='2', markersize=1
    return ax

def calculate_oscr(gt, scores, unk_label=-1):
    """ Calculates the OSCR values, iterating over the score of the target class of every sample,
    produces a pair (ccr, fpr) for every score.
    Args:
        gt (np.array): Integer array of target class labels.
        scores (np.array): Float array of dim [N_samples, N_classes]
        unk_label (int): Label to calculate the fpr, either negatives or unknowns. Defaults to -1 (negatives)
    Returns: Two lists first one for ccr, second for fpr.
    """
    # Change the unk_label to calculate for kn_unknown or unk_unknown
    gt = gt.astype(int)
    #gt = int(gt)
    kn = gt >= 0
    unk = gt == unk_label

    # Get total number of samples of each type
    total_kn = np.sum(kn)
    total_unk = np.sum(unk)

    ccr, fpr = [], []
    # get predicted class for known samples
    pred_class = np.argmax(scores, axis=1)[kn]
    correctly_predicted = pred_class == gt[kn]
    target_score = scores[kn][range(kn.sum()), gt[kn]]

    # get maximum scores for unknown samples
    max_score = np.max(scores, axis=1)[unk]

    # Any max score can be a threshold
    thresholds = np.unique(max_score)

    #print(target_score) #HB
    for tau in thresholds:
        # compute CCR value
        val = (correctly_predicted & (target_score >= tau)).sum() / total_kn
        ccr.append(val)

        val = (max_score >= tau).sum() / total_unk
        fpr.append(val)

    ccr = np.array(ccr)
    fpr = np.array(fpr)
    return ccr, fpr

def plot_oscr(arrays, gt, scale='semilog', title=None, ax_label_font=13, ax=None, unk_label = -1, color = 'r'):
    """Plots OSCR curves for all given scores.
    The scores are stored as arrays: Float array of dim [N_samples, N_classes].
    The arrays contain scores for various loss functions and algorithms as arrays[loss][algorithm].
    """
    print(arrays.items())
    for loss, loss_arrays in arrays.items():
        for algorithm, scores in loss_arrays.items():
            ccr, fpr = calculate_oscr(gt, scores, unk_label)

            ax = plot_single_oscr(fpr, ccr,
                              ax=ax,
                              loss=loss,
                              algorithm=algorithm,
                              scale=scale, color = color)
    if title is not None:
        ax.set_title(title, fontsize=ax_label_font)
    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True, direction='in')
    ax.tick_params(labelbottom=True, labeltop=False, labelleft=True,
                   labelright=False, labelsize=ax_label_font)

    return ax



In [ ]:
#plots all the OSCR curves of one algorithm and loss function combination with the defined titles, colors and values 
def plot_OSCRCharts(benchmark_score_p1, benchmark_ground_truths_p1,benchmark_score_p2, benchmark_ground_truths_p2,benchmark_score_p3, benchmark_ground_truths_p3,
                    scores_p1, ground_truths_p1,scores_p2, ground_truths_p2, scores_p3, ground_truths_p3,
                    scores_p11, ground_truths_p11,scores_p21, ground_truths_p21, scores_p31, ground_truths_p31,
                    scores_p111, ground_truths_p111,scores_p211, ground_truths_p211, scores_p311, ground_truths_p311,
                    scores_p1111, ground_truths_p1111,scores_p2111, ground_truths_p2111, scores_p3111, ground_truths_p3111,
                    scores_p11111, ground_truths_p11111,scores_p21111, ground_truths_p21111, scores_p31111, ground_truths_p31111,
                    title_benchmark, title, title_1, title_11, title_111, title_1111,figsize,
                    algorithm, loss,colorbenchmark = 'b',  color = 'r', color11 = 'g', color111 = 'gray', color1111='orange',color11111='y'):
    fig = pyplot.figure(figsize = (16,10))
    gs = fig.add_gridspec(3,2,hspace = 0.4, wspace = 0.2)
    axs = gs.subplots(sharex = True, sharey = True)
    axs = axs.flat
    font = 10

    protocol_1 = '1'
    protocol_2 = '2'
    protocol_3 = '3'
    legende = []


    #plot the different lines for protocol 1
    if(benchmark_score_p1 != '-'):
      plot_oscr(arrays=benchmark_score_p1[protocol_1], gt=benchmark_ground_truths_p1[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Negative',
                      ax_label_font=font, ax=axs[0], unk_label=-1, color = colorbenchmark)
      plot_oscr(arrays=benchmark_score_p1[protocol_1], gt=benchmark_ground_truths_p1[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Unknown',
                      ax_label_font=font, ax=axs[1], unk_label=-2, color = colorbenchmark)
      legende.append(title_benchmark)

    if(scores_p1 != '-'):
      plot_oscr(arrays=scores_p1[protocol_1], gt=ground_truths_p1[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Negative',
                      ax_label_font=font, ax=axs[0], unk_label=-1, color = color)
      plot_oscr(arrays=scores_p1[protocol_1], gt=ground_truths_p1[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Unknown',
                      ax_label_font=font, ax=axs[1], unk_label=-2, color = color)

      legende.append(title)

    if(scores_p11 != '-'):
      plot_oscr(arrays=scores_p11[protocol_1], gt=ground_truths_p11[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Negative',
                      ax_label_font=font, ax=axs[0], unk_label=-1, color = color11)
      plot_oscr(arrays=scores_p11[protocol_1], gt=ground_truths_p11[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Unknown',
                      ax_label_font=font, ax=axs[1], unk_label=-2, color = color11)

      legende.append(title_1)

    if(scores_p111 != '-'):
      plot_oscr(arrays=scores_p111[protocol_1], gt=ground_truths_p111[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Negative',
                      ax_label_font=font, ax=axs[0], unk_label=-1, color = color111)
      plot_oscr(arrays=scores_p111[protocol_1], gt=ground_truths_p111[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Unknown',
                      ax_label_font=font, ax=axs[1], unk_label=-2, color = color111)

      legende.append(title_11)

    if(scores_p1111 != '-'):
      plot_oscr(arrays=scores_p1111[protocol_1], gt=ground_truths_p1111[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Negative',
                      ax_label_font=font, ax=axs[0], unk_label=-1, color = color1111)
      plot_oscr(arrays=scores_p1111[protocol_1], gt=ground_truths_p1111[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Unknown',
                      ax_label_font=font, ax=axs[1], unk_label=-2, color = color1111)

      legende.append(title_111)
    if(scores_p11111 != '-'):
      plot_oscr(arrays=scores_p11111[protocol_1], gt=ground_truths_p11111[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Negative',
                      ax_label_font=font, ax=axs[0], unk_label=-1, color = color11111)
      plot_oscr(arrays=scores_p11111[protocol_1], gt=ground_truths_p11111[protocol_1], scale="semilog", title=f'$P_{protocol_1}$ Unknown',
                      ax_label_font=font, ax=axs[1], unk_label=-2, color = color11111)

      legende.append(title_1111)


    #plot the different lines for protocol 2
    if(benchmark_score_p2 != '-'):
      plot_oscr(arrays=benchmark_score_p2[protocol_2], gt=benchmark_ground_truths_p2[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Negative',
                      ax_label_font=font, ax=axs[2], unk_label=-1, color = colorbenchmark)
      plot_oscr(arrays=benchmark_score_p2[protocol_2], gt=benchmark_ground_truths_p2[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Unknown',
                      ax_label_font=font, ax=axs[3], unk_label=-2, color = colorbenchmark)

    if(scores_p2 != '-'):
      plot_oscr(arrays=scores_p2[protocol_2], gt=ground_truths_p2[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Negative',
                      ax_label_font=font, ax=axs[2], unk_label=-1, color = color)
      plot_oscr(arrays=scores_p2[protocol_2], gt=ground_truths_p2[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Unknown',
                      ax_label_font=font, ax=axs[3], unk_label=-2, color = color)

    if(scores_p21 != '-'):
      plot_oscr(arrays=scores_p21[protocol_2], gt=ground_truths_p21[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Negative',
                      ax_label_font=font, ax=axs[2], unk_label=-1, color = color11)
      plot_oscr(arrays=scores_p21[protocol_2], gt=ground_truths_p21[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Unknown',
                      ax_label_font=font, ax=axs[3], unk_label=-2, color = color11)

    if(scores_p211 != '-'):
      plot_oscr(arrays=scores_p211[protocol_2], gt=ground_truths_p211[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Negative',
                      ax_label_font=font, ax=axs[2], unk_label=-1, color = color111)
      plot_oscr(arrays=scores_p211[protocol_2], gt=ground_truths_p211[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Unknown',
                      ax_label_font=font, ax=axs[3], unk_label=-2, color = color111)

    if(scores_p2111 != '-'):
      plot_oscr(arrays=scores_p2111[protocol_2], gt=ground_truths_p2111[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Negative',
                      ax_label_font=font, ax=axs[2], unk_label=-1, color = color1111)
      plot_oscr(arrays=scores_p2111[protocol_2], gt=ground_truths_p2111[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Unknown',
                      ax_label_font=font, ax=axs[3], unk_label=-2, color = color1111)
    if(scores_p21111 != '-'):
      plot_oscr(arrays=scores_p21111[protocol_2], gt=ground_truths_p21111[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Negative',
                      ax_label_font=font, ax=axs[2], unk_label=-1, color = color11111)
      plot_oscr(arrays=scores_p21111[protocol_2], gt=ground_truths_p21111[protocol_2], scale="semilog", title=f'$P_{protocol_2}$ Unknown',
                      ax_label_font=font, ax=axs[3], unk_label=-2, color = color11111)



    #plot the different lines for protocol 3
    if(benchmark_score_p3 != '-'):
      plot_oscr(arrays=benchmark_score_p3[protocol_3], gt=benchmark_ground_truths_p3[protocol_3], scale="semilog", title=f'$P_{protocol_2}$ Negative',
                      ax_label_font=font, ax=axs[4], unk_label=-1, color = colorbenchmark)
      plot_oscr(arrays=benchmark_score_p3[protocol_3], gt=benchmark_ground_truths_p3[protocol_3], scale="semilog", title=f'$P_{protocol_2}$ Unknown',
                      ax_label_font=font, ax=axs[5], unk_label=-2, color = colorbenchmark)

    if(scores_p3 != '-'):
      plot_oscr(arrays=scores_p3[protocol_3], gt=ground_truths_p3[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Negative',
                      ax_label_font=font, ax=axs[4], unk_label=-1, color = color)
      plot_oscr(arrays=scores_p3[protocol_3], gt=ground_truths_p3[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Unknown',
                      ax_label_font=font, ax=axs[5], unk_label=-2, color = color)

    if(scores_p31 != '-'):
      plot_oscr(arrays=scores_p31[protocol_3], gt=ground_truths_p31[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Negative',
                      ax_label_font=font, ax=axs[4], unk_label=-1, color = color11)
      plot_oscr(arrays=scores_p31[protocol_3], gt=ground_truths_p31[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Unknown',
                      ax_label_font=font, ax=axs[5], unk_label=-2, color = color11)

    if(scores_p311 != '-'):
      plot_oscr(arrays=scores_p311[protocol_3], gt=ground_truths_p311[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Negative',
                      ax_label_font=font, ax=axs[4], unk_label=-1, color = color111)
      plot_oscr(arrays=scores_p311[protocol_3], gt=ground_truths_p311[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Unknown',
                      ax_label_font=font, ax=axs[5], unk_label=-2, color = color111)

    if(scores_p3111 != '-'):
      plot_oscr(arrays=scores_p3111[protocol_3], gt=ground_truths_p3111[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Negative',
                      ax_label_font=font, ax=axs[4], unk_label=-1, color = color1111)
      plot_oscr(arrays=scores_p3111[protocol_3], gt=ground_truths_p3111[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Unknown',
                      ax_label_font=font, ax=axs[5], unk_label=-2, color = color1111)

    if(scores_p31111 != '-'):
      plot_oscr(arrays=scores_p31111[protocol_3], gt=ground_truths_p31111[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Negative',
                      ax_label_font=font, ax=axs[4], unk_label=-1, color = color11111)
      plot_oscr(arrays=scores_p31111[protocol_3], gt=ground_truths_p31111[protocol_3], scale="semilog", title=f'$P_{protocol_3}$ Unknown',
                      ax_label_font=font, ax=axs[5], unk_label=-2, color = color11111)


    for ax in axs:
        ax.label_outer()
        ax.grid(axis='x', linestyle=':', linewidth=1, color='gainsboro')
        ax.grid(axis='y', linestyle=':', linewidth=1, color='gainsboro')

    # Figure labels
    fig.legend(legende, loc = 'upper center',ncol = (3), bbox_to_anchor = (0.51,0.98))
    fig.text(0.52, 0.05, 'FPR', ha='center', fontsize=font)
    fig.text(0.08, 0.5, 'CCR', va='center', rotation='vertical', fontsize=font)

    return fig

In [ ]:
#define some basics
use_best = False
algorithm = 'threshold'
method = 'mixup'
loss ='entropic'

#default values for the plot when nothing is entered
benchmark_score = '-'
benchmark_ground_truth = ground_truths1 = ground_truths = ground_truths2 = 0
benchmark_score_p1 = benchmark_score_p2 = benchmark_score_p3 = '-'
benchmark_ground_truth_p1 = benchmark_ground_truth_p2 = benchmark_ground_truth_p3 = '-'
scores = '-'
scores1 = '-'
scores2 = '-'
title_benchmark = title = title_1 = title_11 = title_111 = title_1111 = '-'
benchmark_score_p1= benchmark_ground_truth_p1 = '-'
benchmark_score_p2= benchmark_ground_truth_p2 = '-'
benchmark_score_p3= benchmark_ground_truth_p3 = '-'
scores_p1= ground_truths_p1 = '-'
scores_p2= ground_truths_p2 = '-'
scores_p3= ground_truths_p3 = '-'
scores_p11= ground_truths_p11 = '-'
scores_p21= ground_truths_p21 = '-'
scores_p31= ground_truths_p31 = '-'
scores_p111= ground_truths_p111 = '-'
scores_p211= ground_truths_p211 = '-'
scores_p311= ground_truths_p311 = '-'
scores_p1111= ground_truths_p1111 = '-'
scores_p2111= ground_truths_p2111= '-'
scores_p3111= ground_truths_p3111 = '-'
scores_p11111= ground_truths_p11111 = '-'
scores_p21111= ground_truths_p21111= '-'
scores_p31111= ground_truths_p31111 = '-'


#load the scores and ground truths for each method(first entry is for protocol 1, second for protocol 2, thrid for protocol 3)
#comment out the methods you do not need.

#method 1
benchmark_score_p1, benchmark_ground_truth_p1 = load_scores(use_best, algorithm, loss, '1', method,'test', 'benchmark_p1')#blue
benchmark_score_p2, benchmark_ground_truth_p2 = load_scores(use_best, algorithm, loss, '2', method,'test', 'benchmark_p2')
benchmark_score_p3, benchmark_ground_truth_p3 = load_scores(use_best, algorithm, loss, '3', method,'test', 'benchmark_p3')
title_benchmark = 'Benchmark'

#method 2
scores_p1, ground_truths_p1 = load_scores(use_best, algorithm, loss,'1',method,'test', 'p1') #red
scores_p2, ground_truths_p2 = load_scores(use_best, algorithm, loss ,'2',method,'test', 'p2')
scores_p3, ground_truths_p3 = load_scores(use_best, algorithm, loss ,'3',method,'test', 'p3')
title = ''

#method 3
scores_p11, ground_truths_p11 = load_scores(use_best, algorithm, loss,'1',method,'test', 'p1') #yellow
scores_p21, ground_truths_p21 = load_scores(use_best, algorithm, loss ,'2',method,'test', 'p2')
scores_p31, ground_truths_p31 = load_scores(use_best, algorithm, loss ,'3',method,'test', 'p3')
title_1 = ''

#method 4
#scores_p111, ground_truths_p111 = load_scores(use_best, algorithm, loss,'1',method,'test', 'mixup_stage2_p1') #green
#scores_p211, ground_truths_p211 = load_scores(use_best, algorithm, loss ,'2',method,'test', 'mixup_stage2_p2')
#scores_p311, ground_truths_p311 = load_scores(use_best, algorithm, loss ,'3',method,'test', 'mixup_stage2_p3')
#title_11 = 'Mixing-Technique at $S_2$'


#method 5
#scores_p1111, ground_truths_p1111 = load_scores(True, algorithm, loss,'1',method,'test', 'mixupsimple_p1') #orange
#scores_p2111, ground_truths_p2111= load_scores(use_best, algorithm, loss ,'2',method,'test', 'mixupsimple_p2')
#scores_p3111, ground_truths_p3111 = load_scores(use_best, algorithm, loss ,'3',method,'test', 'mixupsimple_p3')
#title_111 = 'Mixing-Technique at $S_2$ + Real Neg. Samples'


#method 6
#scores_p11111, ground_truths_p11111 = load_scores(use_best, algorithm, loss,'1',method,'test', 'mixupmix_p1') #orange
#scores_p21111, ground_truths_p21111= load_scores(use_best, algorithm, loss ,'2',method,'test', 'mixupmix_p2')
#scores_p31111, ground_truths_p31111 = load_scores(use_best, algorithm, loss ,'3',method,'test', 'mixupmix_p3')
#title_1111 = 'Mixing-Technique at $S_2$ + Real Neg. Samples (mixing-style)'

fig = plot_OSCRCharts(benchmark_score_p1, benchmark_ground_truth_p1,benchmark_score_p2, benchmark_ground_truth_p2,benchmark_score_p3, benchmark_ground_truth_p3,
                    scores_p1, ground_truths_p1,scores_p2, ground_truths_p2, scores_p3, ground_truths_p3,
                    scores_p11, ground_truths_p11,scores_p21, ground_truths_p21, scores_p31, ground_truths_p31,
                    scores_p111, ground_truths_p111,scores_p211, ground_truths_p211, scores_p311, ground_truths_p311,
                    scores_p1111, ground_truths_p1111,scores_p2111, ground_truths_p2111, scores_p3111, ground_truths_p3111,
                    scores_p11111, ground_truths_p11111,scores_p21111, ground_truths_p21111, scores_p31111, ground_truths_p31111,
                    title_benchmark, title, title_1, title_11, title_111, title_1111,

                    (16,10), algorithm, loss)


fig.savefig("OSCR.pdf", bbox_inches = 'tight',pad_inches = 0)